### Cohere - Function calls

In [1]:
%pip install "cohere<5" --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cohere, json
API_KEY = os.environ["COHERE_API_KEY"]
co = cohere.Client(API_KEY)

### Sales Example

In [10]:
sales_database = {
    '2023-09-28': {
        'total_sales_amount': 5000,
        'total_units_sold': 100,
    },
    '2023-09-29': {
        'total_sales_amount': 10000,
        'total_units_sold': 250,
    },
    '2023-09-30': {
        'total_sales_amount': 8000,
        'total_units_sold': 200,
    }
}

catalog_sales = {
    'Electronics': {
        'total_sales_amount': 5000,
        'total_units_sold': 100,
    },
    'Clothing': {
        'total_sales_amount': 10000,
        'total_units_sold': 250,
    }
}

product_catalog = {
    'Electronics': [
        {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
        {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
        {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
    ],
    'Clothing': [
        {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
        {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
        {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
    ]
}

In [20]:
def query_daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    report = sales_database.get(day, {})
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}

def query_category_sales(category: str) -> dict:
    report = catalog_sales.get(category, None)
    if report is None:
        return {'category': category, 'summary': "Unknown category or no sales data available"}
    
    return {
            'category': category,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }

def query_product_catalog(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }


functions_map = {
    "query_daily_sales_report": query_daily_sales_report,
    "query_product_catalog": query_product_catalog,
    "query_category_sales": query_category_sales,
}

In [12]:
tools = [
    {
        "name": "query_daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "query_product_catalog",
        "description": "Connects to a product catalog with information about all the products being sold, including categories, prices, and stock levels.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves product information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "query_category_sales",
        "description": "Connects to a category sales catalog to retrieve total sales of a given category.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves sales of the category information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    }
]

In [13]:
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [14]:
message = "Can you provide me products in Electronics category and the sales"

In [15]:
response = co.chat(
    message=message,
    tools=tools,
    preamble=preamble,
    model="command-r"
)

In [23]:
tool_results = []
# Iterate over the tool calls generated by the model
for tool_call in response.tool_calls:
    # here is where you would call the tool recommended by the model, using the parameters recommended by the model
    print(f"Running tool {tool_call.name}, with parameters: {tool_call.parameters}")
    output = functions_map[tool_call.name](**tool_call.parameters)
    # store the output in a list
    outputs = [output]
    print(f">> Tool results: {outputs}")
    # store your tool results in this format
    tool_results.append({
        "call": tool_call,
        "outputs": outputs
    })

Running tool query_product_catalog, with parameters: {'category': 'Electronics'}
>> Tool results: [{'category': 'Electronics', 'products': [{'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20}, {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15}, {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25}]}]
Running tool query_category_sales, with parameters: {'category': 'Electronics'}
>> Tool results: [{'category': 'Electronics', 'summary': 'Total Sales Amount: 5000, Total Units Sold: 100'}]


In [24]:
response = co.chat(
    message=message,
    tools=tools,
    tool_results=tool_results,
    preamble=preamble,
    model="command-r",
    temperature=0.3
)

In [25]:
response.text

'There are many products in the electronics category, such as smartphones (priced at $500), laptops ($1000) and tablets ($300). Currently, the total sales amount in the electronics category is $5000, with 100 units sold in total.'

### SQL Generation

In [41]:
%pip install --quiet langchain langchain_cohere langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

In [17]:
table_def = {
    "user": {
        "user_id": "int primary key",
        "user_name": "str",
        "first_name": "str",
        "last_name": "str"
    },
    "product": {
        "product_id": "int primary key",
        "category_id": "int reference category(category_id)",
        "product_name": "str",
        "price": "float",
        "stock": "int"
    },
    "category": {
        "category_id": "int primary key",
        "category_name": "str"
    },
    "order": {
        "order_id": "int primary key",
        "user_id": "int reference user(user_id)",
        "created_at": "datetime",
    },
    "order_detail": {
        "order_detail_id": "int primary key",
        "order_id": "int reference order(order_id)",
        "product_id": "int reference product(product_id)",
        "price": "float"
    }
}

@tool
def retrieve_table_definition_tool(table_name: str) -> str:
    """Retrieve definition of a given table
    Args:
        table_name(str): Input table name
    """

    definition = table_def.get(table_name, None)
    if definition is None:
        return f"Table {table_name} is not found"
    
    return definition

class retrieve_table_definition_inputs(BaseModel):
    table_name: str = Field(description="Name of table")

retrieve_table_definition_tool.args_schema = retrieve_table_definition_inputs
     

In [47]:
preamble = """
You are an expert who answers the user's question with the most relevant information. You are equipped with a table definition retriever tool, you are smart at splitting the question into multiple table names, please keep the table names in singular form.
"""
message = "Generate SQL query to count number of order of users over time, there are two table 'order' and 'user'"

In [13]:
llm = ChatCohere(model="command-r-plus", temperature=0.3)

In [36]:
prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm=llm,
    tools=[retrieve_table_definition_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[retrieve_table_definition_tool], verbose=True)

In [48]:
response = agent_executor.invoke({
    "input": message,
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will use the retrieve_table_definition_tool to find out the table definitions for the 'order' and 'user' tables. Then, I will write an SQL query to count the number of orders per user over time.
{'tool_name': 'retrieve_table_definition_tool', 'parameters': {'table_name': 'order'}}
{'order_id': 'int primary key', 'user_id': 'int reference user(user_id)', 'created_at': 'datetime'}
{'tool_name': 'retrieve_table_definition_tool', 'parameters': {'table_name': 'user'}}
{'user_id': 'int primary key', 'user_name': 'str', 'first_name': 'str', 'last_name': 'str'}Reflection: I have found the following table definitions:

Order table:
- Order_Id: int primary key
- User_Id: int reference user(user_id)
- Created_At: datetime

User table:
- User_Id: int primary key
- User_Name: str
- First_Name: str
- Last_Name: str

Now, I will write an SQL query to count the number of orders per user over time.
Relevant Documents: 0,1
Cited Documents: 0,1
Answer: Here is a

'Here is an SQL query that counts the number of orders per user over time:\n```sql\nSELECT User_Id, COUNT(Order_Id) AS Order_Count, Created_At\nFROM order\nGROUP BY User_Id, Created_At\nORDER BY User_Id, Created_At;\n```\n\nThis query groups the orders by user and created date, and then counts the number of orders for each user on each date. The results are ordered by user and date.'